In [1]:

!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install rouge_score
!pip install simplet5 -q

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=f7ee02540906d82bbc5adee9e128d75b47436b63b8bdb7947186ccf095886d38
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-api 1.18.0 requires importlib-metadata~=6.0.0, but you have importlib-metadata 6.7.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.11.1 which is incompatible.


In [2]:
!pip install rank_bm25

In [3]:
!pip install sentence-transformers 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=bfb259bafdd0e4138724536780e74ab3297f2da445249a1934871c885ef64421
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np
import pickle

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:

import datasets
answersummData = datasets.load_dataset("alexfabbri/answersumm")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/alexfabbri--answersumm-18f7fb4735693092/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
answersummData

DatasetDict({
    train: Dataset({
        features: ['answers', 'question', 'example_id', 'summaries', 'mismatch_info', 'annotator_id', 'cluster_summaries'],
        num_rows: 2783
    })
    test: Dataset({
        features: ['answers', 'question', 'example_id', 'summaries', 'mismatch_info', 'annotator_id', 'cluster_summaries'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['answers', 'question', 'example_id', 'summaries', 'mismatch_info', 'annotator_id', 'cluster_summaries'],
        num_rows: 500
    })
})

In [7]:
Test_data = answersummData["test"]
Train_data = answersummData["train"]
Val_data = answersummData["validation"]


In [8]:
def flatten(example):
  answers = example["answers"]
#  print(answers[0])
  allanswers = []
  for answer in answers:
    lines = answer["sents"]
    for line in lines:
      allanswers.append(line["text"])
    
  return {
      "question": example["question"]["question"],
      "allanswers": allanswers,
      "firstsummary": example["summaries"][0][0],
      }

In [9]:
Train_data_txt = Train_data.map(flatten, remove_columns=['answers', 'question', 'example_id', 'summaries', 'mismatch_info', 'annotator_id', 'cluster_summaries'])
Val_data_txt = Val_data.map(flatten, remove_columns=['answers', 'question', 'example_id', 'summaries', 'mismatch_info', 'annotator_id', 'cluster_summaries'])
Test_data_txt = Test_data.map(flatten, remove_columns=['answers', 'question', 'example_id', 'summaries', 'mismatch_info', 'annotator_id', 'cluster_summaries'])


  0%|          | 0/2783 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [10]:
"""### **BM25**"""

# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

def answers_tokenized(allanswers):
  tokenized_corpus = []
  for answer in tqdm(allanswers):
    tokenized_corpus.append(bm25_tokenizer(answer))
  return tokenized_corpus


In [11]:
i = 5
query = Val_data_txt[i]["question"]
tokenized_corpus = answers_tokenized(Val_data_txt[i]["allanswers"])
bm25 = BM25Okapi(tokenized_corpus)
bm25_scores = bm25.get_scores(bm25_tokenizer(query))
ranked_ans = bm25.get_top_n(bm25_tokenizer(query), Val_data_txt[i]["allanswers"], n=20)

ranked_ans

Val_data_txt[5]["allanswers"]


  0%|          | 0/45 [00:00<?, ?it/s]

['The best way to approach this is to find someone in your personal network who you could tap for interviews as an interview consultant, or go through a recruiting agency that can do some pre-screening for you.',
 "Since you are small, you don't need to get too sophisticated in the screening as you likely won't attract any heavy hitters, and if you did, they might be bored.",
 'If you go the recruiter route, be sure that they screen for basic competence (online exams, personal interviews, et cetera), then have them send you a pool of candidates that you can screen to see if they are a good fit.',
 'Is there any chance to get a problem set including solutions?',
 'No.',
 'At least not one that would do you any good.',
 'Anything I write here',
 'your applicant could read and it would be meaningless that they easily answer the questions.',
 "Also, that really isn't in scope for this forum.",
 "If you search, you'll find sets of these questions but remember that your applicants can perfor

In [12]:
def ranked_ans_bm25(corpus):
  newcorpus = []
  for i in range(len(corpus)):
    newcorpus_dict = {}
    query = corpus[i]["question"]
    tokenized_corpus = answers_tokenized(corpus[i]["allanswers"])
    bm25 = BM25Okapi(tokenized_corpus)
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    ranked_ans = bm25.get_top_n(bm25_tokenizer(query), corpus[i]["allanswers"], n=20)
    #print('ranked answer ', ranked_ans)
    newcorpus_dict["question"] = query
    newcorpus_dict["rankedanswers"] = ranked_ans
    newcorpus_dict["firstsummary"] = corpus[i]["firstsummary"]
    newcorpus.append(newcorpus_dict)
#    print('corpus rankedanswers ', corpus[i]["rankedanswers"])
  return newcorpus

In [13]:
Train_bm25 = ranked_ans_bm25(Train_data_txt)
Val_bm25 = ranked_ans_bm25(Val_data_txt)
Test_bm25 = ranked_ans_bm25(Test_data_txt)

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

...

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

In [14]:
pickle_out = open("Train-bm25.pickle","wb")
pickle.dump(Train_bm25, pickle_out)
pickle_out.close()

In [15]:
"""## **Bi-Encoder**"""

'## **Bi-Encoder**'

In [16]:
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens

top_k = 20

In [17]:
def ranked_ans_bi_encoder(corpus):
  newcorpus = []
  for i in range(len(corpus)):
    newcorpus_dict={}
    query = corpus[i]["question"]
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    corpus_embeddings = bi_encoder.encode(corpus[i]["allanswers"], convert_to_tensor=True)
    scores = util.dot_score(question_embedding, corpus_embeddings)[0].cuda().tolist()
    doc_score_pairs = list(zip(corpus[i]["allanswers"], scores))
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    i = 0
    ranked_ans = []
    for doc, score in doc_score_pairs:
      if i<20:
        ranked_ans.append(doc)
      i +=1
    #print(len(ranked_ans))    
    #hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    #print(hits)
    #ranked_ans = sorted(corpus[i]["allanswers"], key=lambda x: x['score'], reverse=True)
    newcorpus_dict["question"] = query
    newcorpus_dict["rankedanswers"] = ranked_ans
    newcorpus_dict["firstsummary"] = corpus[i]["firstsummary"]
    newcorpus.append(newcorpus_dict)
    #corpus[i]["allanswers"] = ranked_ans
  return newcorpus

In [18]:
Train_bi_encoder = ranked_ans_bi_encoder(Train_data_txt)
Val_bi_encoder = ranked_ans_bi_encoder(Val_data_txt)
Test_bi_encoder = ranked_ans_bi_encoder(Test_data_txt)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
pickle_out = open("Train-bi-encoder.pickle","wb")
pickle.dump(Train_bi_encoder, pickle_out)
pickle_out.close()




In [20]:
pickle_out = open("Val-bi-encoder.pickle","wb")
pickle.dump(Val_bi_encoder, pickle_out)
pickle_out.close()



In [21]:
pickle_out = open("Test-bi-encoder.pickle","wb")
pickle.dump(Test_bi_encoder, pickle_out)
pickle_out.close()

In [22]:
"""### **Cross-Encoder**"""

'### **Cross-Encoder**'

In [23]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

Downloading:   0%|          | 0.00/794 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [24]:
def ranked_ans_cross_encoder(corpus):
  newcorpus = []
  for i in range(len(corpus)):
    newcorpus_dict={}
    query = corpus[i]["question"]
    cross_inp = [[query, answer] for answer in corpus[i]['allanswers']]
    cross_scores = cross_encoder.predict(cross_inp)
    doc_score_pairs = list(zip(corpus[i]["allanswers"], cross_scores))
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    i = 0
    ranked_ans = []
    for doc, score in doc_score_pairs:
      if i<20:
        ranked_ans.append(doc)
      i +=1
    #print(ranked_ans)
    #print(len(ranked_ans)) 
    #break 
    newcorpus_dict["question"] = query
    newcorpus_dict["rankedanswers"] = ranked_ans
    newcorpus_dict["firstsummary"] = corpus[i]["firstsummary"]
    newcorpus.append(newcorpus_dict)
  return newcorpus

In [25]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util

In [26]:
Train_cross_encoder = ranked_ans_cross_encoder(Train_data_txt)
Val_cross_encoder = ranked_ans_cross_encoder(Val_data_txt)
Test_cross_encoder = ranked_ans_cross_encoder(Test_data_txt)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:

# Write Ranked-Cross-encoder into pickle file

In [28]:
pickle_out = open("Train-cross-encoder.pickle","wb")
pickle.dump(Train_cross_encoder, pickle_out)
pickle_out.close()

In [29]:
pickle_out = open("Val-cross-encoder.pickle","wb")
pickle.dump(Val_cross_encoder, pickle_out)
pickle_out.close()

In [30]:

pickle_out = open("Test-cross-encoder.pickle","wb")
pickle.dump(Test_cross_encoder, pickle_out)
pickle_out.close()